In [1]:
!pip install openpyxl

In [15]:
import pandas as pd
import numpy as np

import torch as t
import torch.nn.functional as f

from tqdm.notebook import trange, tqdm

from bs4 import BeautifulSoup

from transformers import pipeline

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Импорт и обработка данных

---

Создание качественного датасета будет состоять из этапов:
- Загрузим 3 варианта размеченного датасета - объедим все варианты в один и удалим дубликаты
- Очистим данные от html-тегов, шума и малоинформативных предложений(предложения из 1-5 слов убираем)
- Проведем аугментацию данных с более редкими классами
- Для дополнительной балансировки классов и большего разнообразия данных добавим к аугментированным данным данные из **RuSentiment**

In [131]:
data1 = pd.read_excel('/content/sample_data/data_comments_1.xlsx')
data2 = pd.read_excel('/content/sample_data/data_comments_2.xlsx')
data3 = pd.read_excel('/content/sample_data/data_comments_3.xlsx')

In [132]:
data = pd.concat([data1, data2, data3], axis=0).drop_duplicates()

In [133]:
from bs4 import BeautifulSoup

# Удалим html теги
def html_to_text(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = text.replace("\xa0", " ")
    return text.strip()

data.MessageText = data.MessageText.apply(lambda x: html_to_text(x))

In [134]:
import re
from nltk import word_tokenize

# Частично избавимся от "шума"
def is_noise(text):
    if len(word_tokenize(text)) <= 5:  # Убираем слишком короткие строки
        return True
    if re.match(r"^[\W\d_]+$", text):  # Только символы и цифры
        return True
    if re.match(r"^[a-zA-Z]+$", text):  # Только латиница
        return True
    return False

data = data[~data.MessageText.apply(is_noise)]

# Аугментация данных

---

Используем предобученную модель для перефразирования предложений, классы для перефразированных предложений сохраняем

Для того, чтобы избавиться от пересечения примеров в будущем, разделим выборку заранее на тренировочную и тестовую

In [135]:
# Посмотрим на балансировку данных
classes_counts = data.labels.value_counts()
classes_counts

,count
labels,
1,265
2,150
0,64


In [136]:
# Теперь посчитаем сколько примеров нужно сгенерировать для каждого класса для балансировки
classes_samples = np.round(classes_counts.max() / classes_counts).astype(int) - 1
classes_samples

,count
labels,
1,0
2,1
0,3


In [138]:
import torch as t
from transformers import pipeline
from tqdm.notebook import trange, tqdm

paraphrase_model = pipeline(
    "text2text-generation",
    model="cointegrated/rut5-base-paraphraser",
    device=0
)

def paraphrase_text(text, num_return_sequences=3):
    paraphrased = paraphrase_model(
        text,
        max_length=256,
        num_return_sequences=num_return_sequences,
        truncation=True,
        temperature=0.6,
        do_sample=True,
    )

    return [p["generated_text"] for p in paraphrased]

aug_data = {
    'MessageText': [],
    'labels': []
}

for row in classes_samples.to_frame().iterrows():
    class_, return_seqs = row[0], row[1].iloc[0]

    if return_seqs == 0:
        continue

    subdata = data[data.labels == class_]
    for i in trange(subdata.shape[0]):
        aug_text = paraphrase_text(subdata.iloc[i, 0], return_seqs)

        aug_data['MessageText'] += aug_text
        aug_data['labels'] += [subdata.iloc[i, 1]] * return_seqs

        t.cuda.empty_cache()

aug_data = pd.DataFrame(aug_data)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


  0%|          | 0/150 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/64 [00:00<?, ?it/s]

In [140]:
# Объединим данные и избавимся от дубликатов
data = pd.concat([data, aug_data], axis=0).drop_duplicates()

# Рандомно засэмплируем данные
data = data.sample(frac=1)

In [141]:
# Посмотрим на баланс аугментированных данных
classes_counts = data.labels.value_counts()
classes_counts

,count
labels,
2,281
1,265
0,246


# RuSentiment

---

Загрузим данные и объединим их. Из них возьмем часть данных для каждого классаб чтобы размер датасета был приблизительно в 3-3.5 раза больше первоначального. При этом сделаем акцент на отрицательном классе, чтоб будущая модель могла лучше обучиться на разнообразном наборе отрицательных классов

In [142]:
resent1 = pd.read_csv('/content/sample_data/rusentiment_random_posts.csv')
resent2 = pd.read_csv('/content/sample_data/rusentiment_preselected_posts.csv')

rusent_data = pd.concat([resent1, resent2], axis=0).rename(columns={'text': 'MessageText', 'label': 'labels'})

In [144]:
need_classes_samples = {
    'negative': (550, 0),
    'neutral': (500, 1),
    'positive': (450, 2)
}

for cls, (n_samples, idx) in need_classes_samples.items():
    data_sample = rusent_data[rusent_data.labels == cls].sample(frac=1).iloc[:n_samples]
    data_sample.labels = idx

    data = pd.concat([data, data_sample], axis=0)

data = data.sample(frac=1)

# Сбор тестовых данных

---

Объединим примеры данных из 35 сообщений и 100 сообщений

In [162]:
test_data1 = pd.read_excel('/content/sample_data/dataset_comments_35.xlsx')
test_data2 = pd.read_excel('/content/sample_data/dataset_comments_100_test.xlsx')

test_data = pd.concat([test_data1, test_data2], axis=0)[['MessageText', 'labels']]
test_data.MessageText = test_data.MessageText.apply(lambda x: html_to_text(x))

cls2idx = {
    'G': 2,
    'N': 1,
    'B': 0
}

test_data.labels = test_data.labels.apply(lambda x: cls2idx[x])

In [164]:
# Сохраним данные
data.to_excel('/content/train_dataset.xlsx')
test_data.to_excel('/content/test_dataset.xlsx')